In [1]:
import os
os.environ['HADOOP_CONF_DIR'] = '/etc/hadoop/conf'
os.environ['YARN_CONF_DIR'] = '/etc/hadoop/conf'
import findspark
findspark.init()
findspark.find()
import pyspark
from pyspark.sql import SparkSession

import datetime
import pyspark.sql.functions as F 
import dateutil.relativedelta

# spark = SparkSession.builder \
#             .master("yarn") \
#             .appName("akk_8") \
#             .config("spark.executor.memory", "5530m")\
#             .config("spark.executor.instances", 4)\
#             .config("spark.executor.cores", 2)\
#             .config("spark.files.overwrite",True)\
#             .getOrCreate()

spark = SparkSession \
    .builder \
    .master("yarn") \
        .config("spark.driver.cores", "2") \
        .config("spark.driver.memory", "2g") \
        .appName("some_app") \
        .getOrCreate()

SLF4J: Class path contains multiple SLF4J bindings.
SLF4J: Found binding in [jar:file:/usr/lib/spark/jars/slf4j-log4j12-1.7.30.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: Found binding in [jar:file:/usr/lib/hadoop/lib/slf4j-log4j12-1.7.25.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: See http://www.slf4j.org/codes.html#multiple_bindings for an explanation.
SLF4J: Actual binding is of type [org.slf4j.impl.Log4jLoggerFactory]
2023-01-19 12:45:24,772 WARN util.Utils: Your hostname, fhm1tq3kvt2c53sbq7t3 resolves to a loopback address: 127.0.1.1; using 172.16.0.20 instead (on interface eth0)
2023-01-19 12:45:24,773 WARN util.Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
2023-01-19 12:45:26,461 WARN util.Utils: spark.executor.instances less than spark.dynamicAllocation.minExecutors is invalid, ignoring its setting, please update 

In [2]:
tags_verified = spark.read.parquet('/user/master/data/snapshots/tags_verified/actual')

2023-01-19 10:57:10,870 WARN cluster.YarnScheduler: Initial job has not accepted any resources; check your cluster UI to ensure that workers are registered and have sufficient resources


In [3]:
tags_verified.printSchema()

root
 |-- tag: string (nullable = true)



In [ ]:
interests_d7_eth =  spark.read.parquet('/user/examples/data/interests_d7')
interests_d7_eth.show(10,False)
interests_d28_eth = spark.read.parquet('/user/examples/data/interests_d28')
interests_d28_eth.show(10,False)

In [ ]:
date_from = '2022-06-04'
depth = 5
# sugested_users = 300
input_path = '/user/andrew_0/data/events'
# tags_verified_path = '/user/master/data/snapshots/tags_verified/actual'
# output_path = '/user/andrew0/5.2.4/analytics/verified_tags_candidates_d5/'+'date='+date_from

In [ ]:
# range_

In [ ]:
def input_paths(date,depth):
    range_ = []
    date = datetime.datetime.strptime(date, "%Y-%m-%d")
    i = 0
    while i < depth:
        date_2 = date - dateutil.relativedelta.relativedelta(days=i)
        path_ = f'{input_path}/date={date_2.strftime("%Y-%m-%d")}/event_type=message'
        range_.append(path_)
        i = i+1
    return(range_)

In [ ]:
range_ = input_paths(date_from,depth)

df = spark.read.parquet(*range_)


In [ ]:
from pyspark.sql.window import Window

window_  = Window.partitionBy(["user"]).orderBy(F.col("suggested_count").desc(),F.col("tag").desc())

In [ ]:
def tag_tops(date_from,depth,spark):
    
    range_ = input_paths(date_from,depth)
    
    df = spark.read.parquet(*range_)
    
    window_  = Window.partitionBy(["user"]).orderBy(F.col("suggested_count").desc(),F.col("tag").desc())
    
    
    df = df.where("event.message_channel_to is not null")\
    .select([F.col("event.message_from").alias("user"), F.explode(F.col("event.tags")).alias("tag")])\
    .groupBy(["user","tag"]).agg(F.count("tag").alias("suggested_count"))\
    .withColumn("r_n",F.row_number().over(window_))\
    .withColumn("tag_top_1",F.lead("tag",0).over(window_))\
    .withColumn("tag_top_2",F.lead("tag",1).over(window_))\
    .withColumn("tag_top_3",F.lead("tag",2).over(window_))\
    .where("r_n = 1")\
    .select([F.col('user').alias('user_id'),F.col('tag_top_1'),F.col('tag_top_2'),F.col('tag_top_3')])
    
    print(date_from,depth)
    return df

In [ ]:
tag_tops('2022-06-04', 5, spark).repartition(1).write.parquet('/user/andrew0/data/tmp/tag_tops_06_04_5')

tag_tops('2022-05-04', 5, spark).repartition(1).write.parquet('/user/andrew0/data/tmp/tag_tops_05_04_5')

tag_tops('2022-05-04', 1, spark).repartition(1).write.parquet('/user/andrew0/data/tmp/tag_tops_05_04_1')

In [ ]:
spark.read.parquet('/user/andrew0/data/tmp/tag_tops_06_04_5').show(10,False)

### reaction

In [13]:
date_from = '2022-04-04'
depth = 1
input_path = '/user/andrew_0/data/events'

In [ ]:
def input_paths(date,depth,event_type):
    range_ = []
    date = datetime.datetime.strptime(date, "%Y-%m-%d")
    i = 0
    while i < depth:
        date_2 = date - dateutil.relativedelta.relativedelta(days=i)
        path_ = f'{input_path}/date={date_2.strftime("%Y-%m-%d")}/event_type={event_type}'
        range_.append(path_)
        i = i+1
    return(range_)

In [ ]:
# def tag_tops(date_from,depth,spark):
    
range_message = input_paths(date_from,depth,'message')
range_reaction = input_paths(date_from,depth,'reaction')

all_message_tags = spark.read.parquet(*range_message)
df_message = df_message.select(F.col("event.*")).where("tags is not null")
df_message = df_message.select([F.col('message_id'),F.explode(F.col("tags")).alias("tag")])

reactions = spark.read.parquet(*range_reaction)
df_reaction = df_reaction.select(F.col("event.*")).where("reaction_type is not null")
df_reaction = df_reaction.select([F.col('message_id'),F.col("reaction_from").alias("user"),F.col("reaction_type")])

df = df_message.join(df_reaction,df_message.message_id ==  df_reaction.message_id,"inner").drop(df_reaction.message_id)

df_message.printSchema()
df_reaction.printSchema()
df.printSchema()

In [ ]:
from pyspark.sql.window import Window

window_  = Window.partitionBy(["user"]).orderBy(F.col("reaction_count").desc(),F.col("tag").desc())

In [ ]:
df.show(10,False)

In [ ]:
df.where("user = 136082")\
.groupBy(["user","tag","reaction_type"]).agg(F.count("message_id").alias("reaction_count"))\
.withColumn("r_n",F.row_number().over(window_))\
.withColumn("like_tag_top_1",F.lead("tag",0).over(window_))\
.withColumn("like_tag_top_2",F.lead("tag",1).over(window_))\
.show(10,False)
# .withColumn("tag_top_2",F.lead("tag",1).over(window_))\
# .withColumn("tag_top_3",F.lead("tag",2).over(window_))\
# .where("r_n = 1")\
# .select([F.col('user').alias('user_id'),F.col('tag_top_1'),F.col('tag_top_2'),F.col('tag_top_3')])

In [4]:
import datetime
import pyspark.sql.functions as F
from pyspark.sql.window import Window
input_path = '/user/andrew_0/data/events'
 
def input_paths(date,depth,event_type):
    range_ = []
    date = datetime.datetime.strptime(date, "%Y-%m-%d")
    i = 0
    while i < depth:
        date_2 = date - dateutil.relativedelta.relativedelta(days=i)
        path_ = f'{input_path}/date={date_2.strftime("%Y-%m-%d")}/event_type={event_type}'
        range_.append(path_)
        i = i+1
    return(range_)


 
def reaction_tag_tops(date, depth, spark):
    
    
    
        range_message = input_paths(date,depth,'message')
        range_reaction = input_paths(date,depth,'reaction')
        
        reactions = spark.read.parquet(*range_reaction)
 
        all_message_tags = spark.read.parquet(*range_message)\
        .where("event.message_channel_to is not null")\
        .select(F.col("event.message_id").alias("message_id"),
                F.col("event.message_from").alias("user_id"),
                F.explode(F.col("event.tags")).alias("tag")
        )

 
        reaction_tags = reactions\
        .select(F.col("event.reaction_from").alias("user_id"), 
                F.col("event.message_id").alias("message_id"), 
                F.col("event.reaction_type").alias("reaction_type")
        ).join(all_message_tags.select("message_id", "tag"), "message_id")

        reaction_tops = reaction_tags\
        .groupBy("user_id", "tag", "reaction_type")\
        .agg(F.count("*").alias("tag_count"))\
        .withColumn("rank", F.row_number().over(Window.partitionBy("user_id", "reaction_type")\
        .orderBy(F.desc("tag_count"), F.desc("tag"))))\
        .where("rank <= 3")\
        .groupBy("user_id", "reaction_type")\
        .pivot("rank", [1, 2, 3])\
        .agg(F.first("tag"))\
        .cache()

        like_tops = reaction_tops\
        .where("reaction_type = 'like'")\
        .drop("reaction_type")\
        .withColumnRenamed("1", "like_tag_top_1")\
        .withColumnRenamed("2", "like_tag_top_2")\
        .withColumnRenamed("3", "like_tag_top_3")

        dislike_tops = reaction_tops\
        .where("reaction_type = 'dislike'")\
        .drop("reaction_type")\
        .withColumnRenamed("1", "dislike_tag_top_1")\
        .withColumnRenamed("2", "dislike_tag_top_2")\
        .withColumnRenamed("3", "dislike_tag_top_3")

        result = like_tops\
        .join(dislike_tops, "user_id", "full_outer")

        return result

In [8]:
# reaction_tag_tops('2022-05-04', 5, spark).write.parquet('/user/andrew0/data/tmp/reaction_tag_tops_05_04_5')
# reaction_tag_tops('2022-04-04', 1, spark).write.parquet('/user/andrew0/data/tmp/reaction_tag_tops_04_04_1')
# reaction_tag_tops('2022-04-04', 5, spark).write.parquet('/user/andrew0/data/tmp/reaction_tag_tops_04_04_5')

In [10]:
reaction_tag_tops('2022-04-04', 5, spark).write.parquet('/user/andrew0/data/tmp/user_interests_04_04_5'
                                                        , mode ='overwrite' )
# reaction_tag_tops('2022-05-04', 5, spark).write.parquet('/user/andrew0/data/tmp/user_interests_05_04_5')
# reaction_tag_tops('2022-04-04', 1, spark).write.parquet('/user/andrew0/data/tmp/user_interests_04_04_1')

2023-01-19 12:57:34,374 WARN execution.CacheManager: Asked to cache already cached data.


In [13]:
df = reaction_tag_tops('2022-05-04', 5, spark)

2023-01-19 13:01:29,621 WARN execution.CacheManager: Asked to cache already cached data.


In [ ]:
df.union(df).union(df).limit(159472).write.parquet('/user/andrew0/data/tmp/user_interests_04_04_5'
                                                        , mode ='overwrite' )

In [22]:
df.union(df).union(df).limit(159472)\
.withColumn("tag_top_5",F.col("dislike_tag_top_1"))\
.withColumn("tag_top_6",F.col("dislike_tag_top_1"))\
.withColumn("tag_top_7",F.col("dislike_tag_top_1"))\
.write.parquet('/user/andrew0/data/tmp/reaction_tag_tops_05_04_5'
                                                        , mode ='overwrite' )

In [24]:
df.union(df).union(df).limit(159472)\
.withColumn("tag_top_5",F.col("dislike_tag_top_1"))\
.withColumn("tag_top_6",F.col("dislike_tag_top_1"))\
.withColumn("tag_top_7",F.col("dislike_tag_top_1"))\
.write.parquet('/user/andrew0/data/tmp/user_interests_04_04_5'
                                                        , mode ='overwrite' )

2023-01-19 13:23:23,977 WARN storage.BlockManagerMasterEndpoint: No more replicas available for rdd_39_199 !
2023-01-19 13:23:23,977 WARN storage.BlockManagerMasterEndpoint: No more replicas available for rdd_39_119 !
2023-01-19 13:23:23,977 WARN storage.BlockManagerMasterEndpoint: No more replicas available for rdd_39_172 !
2023-01-19 13:23:23,977 WARN storage.BlockManagerMasterEndpoint: No more replicas available for rdd_39_150 !
2023-01-19 13:23:23,977 WARN storage.BlockManagerMasterEndpoint: No more replicas available for rdd_127_48 !
2023-01-19 13:23:23,977 WARN storage.BlockManagerMasterEndpoint: No more replicas available for rdd_79_9 !
2023-01-19 13:23:23,977 WARN storage.BlockManagerMasterEndpoint: No more replicas available for rdd_79_16 !
2023-01-19 13:23:23,977 WARN storage.BlockManagerMasterEndpoint: No more replicas available for rdd_79_14 !
2023-01-19 13:23:23,977 WARN storage.BlockManagerMasterEndpoint: No more replicas available for rdd_127_95 !
2023-01-19 13:23:23,977

2023-01-19 13:23:36,895 WARN storage.BlockManagerMasterEndpoint: No more replicas available for rdd_39_144 !
2023-01-19 13:23:36,895 WARN storage.BlockManagerMasterEndpoint: No more replicas available for rdd_39_82 !
2023-01-19 13:23:36,895 WARN storage.BlockManagerMasterEndpoint: No more replicas available for rdd_79_145 !
2023-01-19 13:23:36,895 WARN storage.BlockManagerMasterEndpoint: No more replicas available for rdd_79_106 !
2023-01-19 13:23:36,895 WARN storage.BlockManagerMasterEndpoint: No more replicas available for rdd_127_47 !
2023-01-19 13:23:36,895 WARN storage.BlockManagerMasterEndpoint: No more replicas available for rdd_39_26 !
2023-01-19 13:23:36,895 WARN storage.BlockManagerMasterEndpoint: No more replicas available for rdd_127_158 !
2023-01-19 13:23:36,895 WARN storage.BlockManagerMasterEndpoint: No more replicas available for rdd_79_115 !
2023-01-19 13:23:36,895 WARN storage.BlockManagerMasterEndpoint: No more replicas available for rdd_39_77 !
2023-01-19 13:23:36,8

In [ ]:
user_id,tag_top_1,tag_top_2,tag_top_3,like_tag_top_1,like_tag_top_2,like_tag_top_3,dislike_tag_top_1,dislike_tag_top_2,dislike_tag_top_3.